In [25]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd

In [26]:
# pdf 파일 저장할 directory 생성
SAVE_DB_DIR = "../data/pdf"
if not os.path.exists(SAVE_DB_DIR):
    os.makedirs(SAVE_DB_DIR)

In [32]:
url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
response = requests.get(url, headers=headers)
print(response)

ConnectionError: HTTPConnectionPool(host='www.bok.or.kr', port=80): Max retries exceeded with url: /portal/bbs/B0000245/list.do?menuNo=200761 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000017BED75F820>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [27]:
for i in range(3, 33):
    b_url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
    params = {
    'pageIndex' : i
    }
    resp = requests.get(b_url, params = params)
    soup = BeautifulSoup(resp.content)
    
    # 하위 수준의 li를 제거하기 위한 코드
    # 첫번째 li를 find로 찾은 후 sibling으로 형제수준만 뽑아냄
    li_list = []
    li= soup.find('div', class_='bdLine type2').find('ul').find('li')
    li_li = li.find_next_siblings('li')
    li_list.append(li)
    li_list.extend(li_li)
    
print(li_list)

ConnectionError: HTTPConnectionPool(host='www.bok.or.kr', port=80): Max retries exceeded with url: /portal/bbs/B0000245/list.do?menuNo=200761&pageIndex=3 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000017BEC91F700>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [69]:
li_list = []

# 2008~2021년 금통위 의사록 크롤링
for i in range(3, 33):
    b_url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
    params = {
    'pageIndex' : i
    }
    resp = requests.get(b_url, params = params)
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    # 하위 수준의 li를 제거하기 위한 코드
    # 첫번째 li를 find로 찾은 후 sibling으로 형제수준만 뽑아냄
    
    # li= soup.find('div', class_='bdLine type2').find('ul').find('li')
    titles = soup.select('.col.m10.s10.x9.ctBx')
    for title in titles:
        year = title.select_one('.titlesub').text.split('(')[1][:4]
        if 2008 <= int(year) <= 2021:
            li_list.append(title.text.strip())
    # li_li = li.find_next_siblings('li')
    # li_list.append(li)
    # li_list.extend(li_li)
    
    # print(i)

    # pdf파일 만들기
    result_list = []
    try:
        for x in range(0, len(li_list)):
            new_dict = {}
            link_li = li_list[x].find('div', class_='fileGoupBox').find('ul').find_all('li')

            # 05년 이전 데이터는 pdf파일이 존재하지 않아, 오류
            # 05년 부터의 데이터는 pdf와 hwp 파일 모두 존재
            # 두 파일의 순서가 제각각이라 if문을 이용하여 pdf파일만 선택함
            for link in link_li:

                if link.find('a').attrs['title'][-3:] == 'pdf':
                    link_u = link.find('a').attrs['href']
                    title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
                    url2 = 'http://www.bok.or.kr' + link_u

                    # 다운로드 링크를 requests.get을 사용하여 텍스트 출력
                    # 추후 pdf가 아닌 txt형식으로 저장하여 다음 단계를 생략할 수 있는지 여부 확인 필
                    file_res2 = requests.get(url2)
                    file_name = '{}.pdf'.format(title)

                    txt_name = os.path.join(SAVE_DB_DIR, file_name)

                    with open(txt_name, 'wb') as f:
                        f.write(file_res2.content)
    except:
        print(f"get url.content error and pass page{i}, {x}")

['금융통화위원회 의사록(2021년도 제25차)(2021.12.23.)',
 '금융통화위원회 의사록(2021년도 제24차)(2021.12.9.)',
 '금융통화위원회 의사록(2021년도 제23차)(2021.11.25.)',
 '금융통화위원회 의사록(2021년도 제20차)(2021.10.12.)',
 '금융통화위원회 의사록(2021년도 제19차)(2021.9.24.)',
 '금융통화위원회 의사록(2021년도 제18차)(2021.9.9.)',
 '금융통화위원회 의사록(2021년도 제17차)(2021.8.26.)',
 '금융통화위원회 의사록(2021년도 제15차)(2021.7.29.)',
 '금융통화위원회 의사록(2021년도 제14차)(2021.7.15.)',
 '금융통화위원회 의사록(2021년도 제13차)(2021.6.30.)',
 '금융통화위원회 의사록(2021년도 제12차)(2021.6.22.)',
 '금융통화위원회 의사록(2021년도 제11차)(2021.6.10)',
 '금융통화위원회 의사록(2021년도 제10차)(2021.5.27)',
 '금융통화위원회 의사록(2021년도 제7차)(2021.4.15)',
 '금융통화위원회 의사록(2021년도 제6차)(2021.3.25)',
 '금융통화위원회 의사록(2021년도 제5차)(2021.3.11)',
 '금융통화위원회 의사록(2021년도 제4차)(2021.2.25)',
 '금융통화위원회 의사록(2021년도 제2차)(2021.1.28.)',
 '금융통화위원회 의사록(2021년도 제1차)(2021.1.15.)',
 '금융통화위원회 의사록(2020년도 제27차)(2020.12.24)',
 '금융통화위원회 의사록(2020년도 제26차)(2020.12.10)',
 '금융통화위원회 의사록(2020년도 제25차)(2020.11.26)',
 '금융통화위원회 의사록(2020년도 제23차)(2020.10.29)',
 '금융통화위원회 의사록(2020년도 제22차)(2020.10.14)',
 '금융통화위원회 의사록(2020년도 제21차)

In [24]:
li_list = []

# 2008~2021년 금통위 의사록 크롤링
for i in range(3, 33):
    b_url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
    params = {
    'pageIndex' : i
    }
    resp = requests.get(b_url, params = params)
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    # 하위 수준의 li를 제거하기 위한 코드
    # 첫번째 li를 find로 찾은 후 sibling으로 형제수준만 뽑아냄
    
    # li= soup.find('div', class_='bdLine type2').find('ul').find('li')
    titles = soup.select('.bdLine type2 > ul')
    print(titles)
#     for title in titles:
#         year = title.select_one('.titlesub').text.split('(')[1][:4]
#         if 2008 <= int(year) <= 2021:
#             li_list.append(title)
#     # li_li = li.find_next_siblings('li')
#     # li_list.append(li)
#     # li_list.extend(li_li)
    
#     # print(i)
# li_list

ConnectionError: HTTPConnectionPool(host='www.bok.or.kr', port=80): Max retries exceeded with url: /portal/bbs/B0000245/list.do?menuNo=200761&pageIndex=3 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000017BEB66CDF0>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [16]:
li_list

[<a class="secretSet" href="/portal/bbs/B0000245/view.do?nttId=10068377&amp;menuNo=200761&amp;pageIndex=3">
 <span class="title">
 <span class="titlesub">금융통화위원회 의사록(2021년도 제25차)(2021.12.23.)</span>
 </span>
 </a>,
 <a class="secretSet" href="/portal/bbs/B0000245/view.do?nttId=10068169&amp;menuNo=200761&amp;pageIndex=4">
 <span class="title">
 <span class="titlesub">금융통화위원회 의사록(2021년도 제24차)(2021.12.9.)</span>
 </span>
 </a>,
 <a class="secretSet" href="/portal/bbs/B0000245/view.do?nttId=10067916&amp;menuNo=200761&amp;pageIndex=4">
 <span class="title">
 <span class="titlesub">금융통화위원회 의사록(2021년도 제23차)(2021.11.25.)</span>
 </span>
 </a>,
 <a class="secretSet" href="/portal/bbs/B0000245/view.do?nttId=10067287&amp;menuNo=200761&amp;pageIndex=4">
 <span class="title">
 <span class="titlesub">금융통화위원회 의사록(2021년도 제20차)(2021.10.12.)</span>
 </span>
 </a>,
 <a class="secretSet" href="/portal/bbs/B0000245/view.do?nttId=10066931&amp;menuNo=200761&amp;pageIndex=4">
 <span class="title">
 <span clas

In [20]:
 # pdf파일 만들기
result_list = []
for x in range(0, len(li_list)):
    new_dict = {}
    link_li = li_list[x].select('.fileGoupBox') #.find('ul').find_all('li')
    print(link_li)

        #     # 05년 이전 데이터는 pdf파일이 존재하지 않아, 오류
        #     # 05년 부터의 데이터는 pdf와 hwp 파일 모두 존재
        #     # 두 파일의 순서가 제각각이라 if문을 이용하여 pdf파일만 선택함
        # for link in link_li:

        #     if link.find('a').attrs['title'][-3:] == 'pdf':
        #         link_u = link.find('a').attrs['href']
        #         title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
        #         url2 = 'http://www.bok.or.kr' + link_u

        #             # 다운로드 링크를 requests.get을 사용하여 텍스트 출력
        #             # 추후 pdf가 아닌 txt형식으로 저장하여 다음 단계를 생략할 수 있는지 여부 확인 필
        #         file_res2 = requests.get(url2)
        #         file_name = '{}.pdf'.format(title)

        #         txt_name = os.path.join(SAVE_DB_DIR, file_name)

        #         with open(txt_name, 'wb') as f:
        #             f.write(file_res2.content)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
